# Creating a basic Neural Networks
In this notebook I using tensorflow, to create a DNN with subclassing API.

The data is sintetic, only to understand how the library works...

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

import warnings

warnings.filterwarnings('ignore')

2024-05-26 19:01:05.548374: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-26 19:01:05.577668: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 19:01:05.995401: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def sintetic_data(shape:tuple):
    X = tf.random.uniform(shape=shape)
    y= tf.random.normal(shape=shape)
    return X,y 

X_train,y_train = sintetic_data(shape = (100,1))
X_val,y_val = sintetic_data(shape = (20,1))
X_test,y_test = sintetic_data(shape = (10,1))

2024-05-26 19:01:06.516830: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-26 19:01:06.542630: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-26 19:01:06.545391: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [21]:

class BasicModel(tf.keras.Model):
    def __init__(self,units, activation,num_hidden_states,**kwargs):
        super().__init__(**kwargs)
        self.num_hidden_states=num_hidden_states
        self.input_layer = tf.keras.Input(shape=(1,))
        self.dense1 = tf.keras.layers.Dense(units, activation=activation,input_shape = [1])
        self.dense2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
    
    def call(self, inputs):
        dense1 = self.dense1(inputs)
        dense2 = self.dense2(dense1)
        for i in range(self.num_hidden_states - 1):
            dense2 = self.dense2(dense2)
        output = self.main_output(dense2)
        return output
    
model = BasicModel(units=100, activation="relu",num_hidden_states=2)

loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss=loss_fn,
              optimizer=optimizer,
              metrics=["mae"])
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=100,verbose=False)

In [22]:
model.evaluate(X_test,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 1.3157 - mae: 0.8310


[1.3157087564468384, 0.8310014009475708]

In [23]:
model.predict(tf.random.uniform(shape=(10,1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


array([[0.04681226],
       [0.04688554],
       [0.09411073],
       [0.04809505],
       [0.04922621],
       [0.05027952],
       [0.07341749],
       [0.09323829],
       [0.04922621],
       [0.08973147]], dtype=float32)

In [33]:
def build_model(hp):  
  hp_units = hp.Int('units', min_value=4, max_value=128, step=4)  # Search for units between 4 and 128 in steps of 4
  hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'elu'])  # Search for these activation functions
  hp_num_hidden_states = hp.Int('num_hidden_states', min_value=2, max_value=5)  # Search for number of hidden layers (1 to 5)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  with tf.device('/GPU:0'):
    model = BasicModel(units=hp_units, activation=hp_activation, num_hidden_states=hp_num_hidden_states)
    
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='mse', metrics=['mae'])  # Compile with desired optimizer, loss, and metrics

  return model

In [37]:
tuner = kt.RandomSearch(build_model,
                     objective=['mae'],
                     max_trials= 10,
                     directory='tuning',
                     project_name='basic_model',
                     )

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

In [38]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val),verbose=True)

Trial 10 Complete [00h 00m 04s]
multi_objective: 0.8054307699203491

Best multi_objective So Far: 0.7961602807044983
Total elapsed time: 00h 00m 49s


In [39]:
best_models = tuner.get_best_models(num_models=2)  

best_model = best_models[0]
best_model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 1.6732 - mae: 1.0105


[1.673233985900879, 1.0104600191116333]

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]